# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline

### 1. Import libraries and load data from database.
* Import Python libraries
* Load dataset from database with read_sql_table
* Define feature and target variables X and Y

In [1]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Alberto\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Alberto\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Alberto\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
# Import libraries 
import numpy as np 
import pandas as pd
pd.set_option('display.max_columns', 500)

import sys
import os
import re
from sqlalchemy import create_engine
import pickle

from scipy.stats import gmean

# import relevant functions/modules from the sklearn
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.base import BaseEstimator,TransformerMixin

In [3]:
# Load data from database
database_filepath = '../data/disaster_response_db.db'
engine = create_engine('sqlite:///' + database_filepath)
table_name = 'T_disaster_df'
df = pd.read_sql_table(table_name, engine)

In [4]:
df.describe()

,id,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,water,food,shelter,clothing,money,missing_people,refugees,death,other_aid,infrastructure_related,transport,buildings,electricity,tools,hospitals,shops,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
count,26215.000000,26215.000000,26215.000000,26215.000000,26215.000000,26215.000000,26215.000000,26215.000000,26215.000000,26215.000000,26215.0,26215.000000,26215.000000,26215.000000,26215.000000,26215.000000,26215.000000,26215.000000,26215.000000,26215.000000,26215.000000,26215.000000,26215.000000,26215.000000,26215.000000,26215.000000,26215.000000,26215.000000,26215.000000,26215.000000,26215.000000,26215.000000,26215.000000,26215.000000,26215.000000,26215.000000,26215.000000
mean,15224.871333,0.766470,0.170666,0.004501,0.414267,0.079496,0.050086,0.027618,0.017967,0.032806,0.0,0.063780,0.111501,0.088270,0.015449,0.023040,0.011368,0.033378,0.045546,0.131451,0.065039,0.045813,0.050849,0.020294,0.006065,0.010795,0.004578,0.011787,0.043906,0.278352,0.082205,0.093191,0.010757,0.093649,0.020217,0.052489,0.193591
std,8827.053788,0.423085,0.376224,0.066941,0.492604,0.270517,0.218126,0.163878,0.132833,0.178131,0.0,0.244366,0.314758,0.283693,0.123333,0.150034,0.106013,0.179625,0.208503,0.337900,0.246600,0.209084,0.219693,0.141006,0.077645,0.103340,0.067504,0.107929,0.204890,0.448196,0.274682,0.290705,0.103160,0.291345,0.140746,0.223015,0.395120
min,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,7446.500000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,15663.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,22924.500000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,30265.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [5]:
# Remove child alone, column of 0's
df = df.drop(['child_alone'], axis = 1)

In [6]:
df.groupby("related").count()

,id,message,original,genre,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,water,food,shelter,clothing,money,missing_people,refugees,death,other_aid,infrastructure_related,transport,buildings,electricity,tools,hospitals,shops,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
related,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,6122,6122,3395,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122
1,20093,20093,6775,20093,20093,20093,20093,20093,20093,20093,20093,20093,20093,20093,20093,20093,20093,20093,20093,20093,20093,20093,20093,20093,20093,20093,20093,20093,20093,20093,20093,20093,20093,20093,20093,20093,20093,20093


In [7]:
# Extract X and y variables from the data for modelling
X = df['message']
y = df.iloc[:, 4:]

### 2. Write a tokenization function to process your text data


In [8]:
def tokenize(text, url_place_holder_string = 'urlplaceholder'):
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    
    # Replace all urls with a urlplaceholder string
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    
    # Extract all the urls from the provided text 
    detected_urls = re.findall(url_regex, text)
    
    # Replace url with a url placeholder string
    for detected_url in detected_urls:
        text = text.replace(detected_url, url_place_holder_string)

    # Extract the word tokens from the provided text
    tokens = nltk.word_tokenize(text)
    
    #Lemmanitizer to remove inflectional and derivationally related forms of a word
    lemmatizer = nltk.WordNetLemmatizer()

    # List of clean tokens
    clean_tokens = [lemmatizer.lemmatize(w).lower().strip() for w in tokens]
    return clean_tokens
    

In [9]:
# Build a custom transformer which will extract the starting verb of a sentence
class StartingVerbExtractor(BaseEstimator, TransformerMixin):
    """
    Starting Verb Extractor class
    
    This class extract the starting verb of a sentence,
    creating a new feature for the ML classifier
    """

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    # Given it is a tranformer we can return the self 
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

### 3. Build a machine learning pipeline
This machine pipeline should take in the message column as input and output classification results on the other 36 categories in the dataset. You may finde the MultiOutputClassifier helpful for predicting multiple target variables.

In [10]:
pipeline1 = Pipeline([
    ('features', FeatureUnion([
            ('text_pipeline', Pipeline([
                ('count_vectorizer', CountVectorizer(tokenizer=tokenize)),
                ('tfidf_transformer', TfidfTransformer())
            ]))
    ])),
    ('classifier', MultiOutputClassifier(AdaBoostClassifier()))
])

pipeline2 = Pipeline([
        ('features', FeatureUnion([

                ('text_pipeline', Pipeline([
                    ('count_vectorizer', CountVectorizer(tokenizer=tokenize)),
                    ('tfidf_transformer', TfidfTransformer())
            ])),

            ('starting_verb_transformer', StartingVerbExtractor())
        ])),

        ('classifier', MultiOutputClassifier(AdaBoostClassifier()))
    ])


### 4. Train pipeline
* Split data into train and test sets
* Train pipeline

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
pipeline_fitted = pipeline1.fit(X_train, y_train)

### 5. Test your model

report the F1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's classification_report on each.

In [12]:
y_prediction_train = pipeline_fitted.predict(X_train)
y_prediction_test = pipeline_fitted.predict(X_test)

In [13]:
print(classification_report(y_test.values, y_prediction_test, target_names=y.columns.values))

                        precision    recall  f1-score   support

               related       0.83      0.94      0.88      5041
               request       0.78      0.53      0.63      1159
                 offer       0.00      0.00      0.00        23
           aid_related       0.76      0.61      0.68      2761
          medical_help       0.64      0.28      0.39       528
      medical_products       0.67      0.32      0.43       356
     search_and_rescue       0.68      0.22      0.33       192
              security       0.16      0.03      0.06       117
              military       0.66      0.33      0.44       209
                 water       0.71      0.66      0.68       400
                  food       0.82      0.70      0.76       772
               shelter       0.83      0.52      0.64       631
              clothing       0.69      0.40      0.51       103
                 money       0.47      0.28      0.35       154
        missing_people       0.56      

D:\Python\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\Python\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [14]:
print('\n',classification_report(y_train.values, y_prediction_train, target_names=y.columns.values))


                         precision    recall  f1-score   support

               related       0.83      0.94      0.88     15052
               request       0.80      0.56      0.66      3315
                 offer       0.38      0.08      0.14        95
           aid_related       0.77      0.61      0.68      8099
          medical_help       0.66      0.27      0.38      1556
      medical_products       0.68      0.34      0.46       957
     search_and_rescue       0.67      0.23      0.35       532
              security       0.38      0.06      0.10       354
              military       0.70      0.36      0.47       651
                 water       0.76      0.67      0.71      1272
                  food       0.81      0.71      0.76      2151
               shelter       0.78      0.54      0.64      1683
              clothing       0.80      0.48      0.60       302
                 money       0.66      0.34      0.45       450
        missing_people       0.70    

### 6. Improve your model
Use grid search to find better parameters

In [15]:
# pipeline1.get_params().keys()
parameters_grid = {'classifier__estimator__learning_rate': [0.01, 0.02, 0.05],
              'classifier__estimator__n_estimators': [10, 20, 40]}

#parameters_grid = {'classifier__estimator__n_estimators': [10, 20, 40]}

cv = GridSearchCV(pipeline1, param_grid=parameters_grid, scoring='f1_micro', n_jobs=-1)
cv.fit(X_train, y_train)

GridSearchCV(estimator=Pipeline(steps=[('features',
                                        FeatureUnion(transformer_list=[('text_pipeline',
                                                                        Pipeline(steps=[('count_vectorizer',
                                                                                         CountVectorizer(tokenizer=<function tokenize at 0x0000021C90AAB820>)),
                                                                                        ('tfidf_transformer',
                                                                                         TfidfTransformer())]))])),
                                       ('classifier',
                                        MultiOutputClassifier(estimator=AdaBoostClassifier()))]),
             n_jobs=-1,
             param_grid={'classifier__estimator__learning_rate': [0.01, 0.02,
                                                                  0.05],
                         'classifier_

# 7. Test your model
Show the accuracy, precision, and recall of the tuned model.

Since this project focuses on code quality, process, and pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [16]:
# Print the classification report on test data
print(classification_report(y_test.values, y_prediction_test, target_names=y.columns.values))

                        precision    recall  f1-score   support

               related       0.83      0.94      0.88      5041
               request       0.78      0.53      0.63      1159
                 offer       0.00      0.00      0.00        23
           aid_related       0.76      0.61      0.68      2761
          medical_help       0.64      0.28      0.39       528
      medical_products       0.67      0.32      0.43       356
     search_and_rescue       0.68      0.22      0.33       192
              security       0.16      0.03      0.06       117
              military       0.66      0.33      0.44       209
                 water       0.71      0.66      0.68       400
                  food       0.82      0.70      0.76       772
               shelter       0.83      0.52      0.64       631
              clothing       0.69      0.40      0.51       103
                 money       0.47      0.28      0.35       154
        missing_people       0.56      

D:\Python\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\Python\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [17]:

# Print classification report on training data
print('\n',classification_report(y_train.values, y_prediction_train, target_names=y.columns.values))


                         precision    recall  f1-score   support

               related       0.83      0.94      0.88     15052
               request       0.80      0.56      0.66      3315
                 offer       0.38      0.08      0.14        95
           aid_related       0.77      0.61      0.68      8099
          medical_help       0.66      0.27      0.38      1556
      medical_products       0.68      0.34      0.46       957
     search_and_rescue       0.67      0.23      0.35       532
              security       0.38      0.06      0.10       354
              military       0.70      0.36      0.47       651
                 water       0.76      0.67      0.71      1272
                  food       0.81      0.71      0.76      2151
               shelter       0.78      0.54      0.64      1683
              clothing       0.80      0.48      0.60       302
                 money       0.66      0.34      0.45       450
        missing_people       0.70    

In [18]:
#Use pipeline2 which includes StartingVerbEstimator
X_train, X_test, y_train, y_test = train_test_split(X, y)
pipeline_fitted = pipeline2.fit(X_train, y_train)

y_prediction_train = pipeline_fitted.predict(X_train)
y_prediction_test = pipeline_fitted.predict(X_test)

# Print classification report on test data
print(classification_report(y_test.values, y_prediction_test, target_names=y.columns.values))

                        precision    recall  f1-score   support

               related       0.82      0.94      0.88      4995
               request       0.74      0.55      0.63      1113
                 offer       0.00      0.00      0.00        33
           aid_related       0.75      0.60      0.67      2647
          medical_help       0.57      0.26      0.35       489
      medical_products       0.61      0.32      0.42       310
     search_and_rescue       0.67      0.18      0.29       175
              security       0.14      0.02      0.04       127
              military       0.62      0.35      0.44       208
                 water       0.72      0.66      0.69       407
                  food       0.77      0.71      0.74       715
               shelter       0.76      0.53      0.62       598
              clothing       0.66      0.29      0.40       107
                 money       0.53      0.26      0.35       151
        missing_people       0.64      

D:\Python\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\Python\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [19]:
# Print classification report on training data
print('\n',classification_report(y_train.values, y_prediction_train, target_names=y.columns.values))


                         precision    recall  f1-score   support

               related       0.83      0.95      0.88     15098
               request       0.80      0.55      0.65      3361
                 offer       0.42      0.13      0.20        85
           aid_related       0.77      0.62      0.69      8213
          medical_help       0.67      0.30      0.42      1595
      medical_products       0.71      0.36      0.48      1003
     search_and_rescue       0.69      0.21      0.32       549
              security       0.42      0.07      0.12       344
              military       0.70      0.41      0.52       652
                 water       0.77      0.66      0.71      1265
                  food       0.82      0.71      0.76      2208
               shelter       0.80      0.56      0.66      1716
              clothing       0.81      0.46      0.58       298
                 money       0.60      0.32      0.42       453
        missing_people       0.70    

In [20]:
m = pickle.dumps('classifier.pkl')